In [ ]:
!pip install langchain langchain-community langchain-google-genai faiss-cpu

## 색인하기

In [ ]:
from langchain_core.documents import Document
from langchain_community.vectorstores import FAISS
from langchain_google_genai import GoogleGenerativeAIEmbeddings
import os
import json
import glob
import requests

In [ ]:
# API 키 설정
os.environ["GOOGLE_API_KEY"] = "your-api-key"

GitHub에 있는 판례 데이터 파일을 다운로드 받아서 색인

In [ ]:
# GitHub에서 precedent_sample 디렉토리의 JSON 파일들 다운로드
GITHUB_API_URL = "https://api.github.com/repos/nongaussian/class-2026-lginnotek-llm/contents/precedent_sample"
RAW_BASE_URL = "https://raw.githubusercontent.com/nongaussian/class-2026-lginnotek-llm/main/precedent_sample"
LOCAL_DIR = "precedent_sample"

In [ ]:
# 로컬 디렉토리 생성 (없으면)
os.makedirs(LOCAL_DIR, exist_ok=True)
print(f"📁 '{LOCAL_DIR}' 디렉토리 준비 완료")

# GitHub API로 파일 목록 가져오기
print("🔍 GitHub에서 파일 목록을 가져오는 중...")
response = requests.get(GITHUB_API_URL)
if response.status_code != 200:
    raise Exception(f"GitHub API 요청 실패: {response.status_code}")

files = response.json()
json_files_info = [f for f in files if f["name"].endswith(".json")]
print(f"📂 GitHub에서 {len(json_files_info)}개의 JSON 파일을 발견했습니다.")

# 각 JSON 파일 다운로드
for file_info in json_files_info:
    file_name = file_info["name"]
    local_path = os.path.join(LOCAL_DIR, file_name)

    # 파일 다운로드
    raw_url = f"{RAW_BASE_URL}/{file_name}"
    file_response = requests.get(raw_url)

    if file_response.status_code == 200:
        with open(local_path, "w", encoding="utf-8") as f:
            f.write(file_response.text)
        print(f"  ✓ {file_name} 다운로드 완료")
    else:
        print(f"  ✗ {file_name} 다운로드 실패: {file_response.status_code}")

# 로컬 디렉토리에서 JSON 파일들 로드
json_files = glob.glob(f"{LOCAL_DIR}/*.json")
print(f"\n📂 {len(json_files)}개의 JSON 파일을 로드합니다.")

In [ ]:
!ls

JSON 형식의 판례 데이터 파일을 파싱하여 벡터 색인을 위해 준비 (문서마다 Document 객체 생성)

In [ ]:
docs = []
for file_path in json_files:
    with open(file_path, "r", encoding="utf-8") as f:
        data = json.load(f)

    # 판례 정보 추출
    info = data.get("info", {})
    case_no = info.get("caseNo", "")
    case_nm = info.get("caseNm", "")
    court_nm = info.get("courtNm", "")
    judmn_date = info.get("judmnAdjuDe", "")

    # 본문 내용 구성 (사실관계 + 법원 판단 + 결론)
    facts = data.get("facts", {}).get("bsisFacts", [])
    dcss = data.get("dcss", {}).get("courtDcss", [])
    cnclsns = data.get("close", {}).get("cnclsns", [])

    content = f"[사건번호: {case_no}] {case_nm}\n"
    content += f"법원: {court_nm} | 선고일: {judmn_date}\n\n"
    content += "[사실관계]\n" + "\n".join(facts) + "\n\n"
    content += "[법원 판단]\n" + "\n".join(dcss) + "\n\n"
    content += "[결론]\n" + "\n".join(cnclsns)

    # 메타데이터와 함께 Document 생성
    doc = Document(
        page_content=content,
        metadata={
            "case_no": case_no,
            "case_nm": case_nm,
            "court_nm": court_nm,
            "judmn_date": judmn_date,
            "source": file_path
        }
    )
    docs.append(doc)

print(f"📄 {len(docs)}개의 문서를 생성했습니다.")

임베딩 모델을 기반으로 각 문서를 벡터로 표현 & FAISS 색인을 생성하여 로컬의 precedent_index 폴더에 저장

In [ ]:
# 2. 임베딩 모델 설정 (Google Generative AI)
embeddings = GoogleGenerativeAIEmbeddings(model="models/gemini-embedding-001")

print("⏳ 문서 벡터화 진행 중...")

# 3. FAISS 인덱스 생성 (메모리 상에 구축)
db = FAISS.from_documents(docs, embeddings)

# 4. 로컬 디스크에 저장 (핵심!)
# 실행 경로에 'precedent_index'라는 폴더가 생성됩니다.
db.save_local("precedent_index")

print("✅ 인덱스 저장 완료! ('precedent_index' 폴더 확인)")

In [ ]:
!ls

## 검색하기

In [ ]:
from langchain_community.vectorstores import FAISS
from langchain_google_genai import GoogleGenerativeAIEmbeddings, ChatGoogleGenerativeAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
import os

판례 인덱스 DB를 불러오기 (RAG 검색에 사용할 임베딩 모델은 색인시 사용한 모델과 같은 모델을 사용해야 함)

In [ ]:
# 1. 저장된 판례 인덱스 불러오기
# rag-build-index.py와 동일한 임베딩 모델 사용
embeddings = GoogleGenerativeAIEmbeddings(model="models/gemini-embedding-001")

# 'allow_dangerous_deserialization=True'는 신뢰할 수 있는 로컬 파일일 경우에만 사용합니다.
new_db = FAISS.load_local(
    "precedent_index",
    embeddings,
    allow_dangerous_deserialization=True
)

# 2. 검색기(Retriever)로 변환 - 유사한 판례 3개 검색
retriever = new_db.as_retriever(search_kwargs={"k": 3})

# 3. LLM 설정 (Gemini 2.0 Flash)
llm = ChatGoogleGenerativeAI(model="gemini-flash-lite-latest", temperature=0)

프롬프트 템플릿 준비

In [ ]:
# 4. 프롬프트 템플릿 정의 (법률 상담용)
prompt = ChatPromptTemplate.from_template("""
당신은 법률 전문가입니다. 아래 제공된 판례들을 참고하여 질문에 답변해주세요.

[참고 판례]
{context}

[질문]
{question}

[답변 지침]
1. 참고 판례의 사실관계와 질문의 상황을 비교 분석하세요.
2. 관련 판례의 법원 판단을 근거로 답변하세요.
3. 참고한 판례의 사건번호를 명시하세요.
4. 실제 법적 조언이 아닌 참고용 정보임을 명시하세요.

답변:
""")

체인 구성

In [ ]:
# 5. 문서 포맷팅 함수
def format_docs(docs):
    #print(docs)
    return "\n\n".join(doc.page_content for doc in docs)

# 6. LCEL 체인 구성
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

체인 실행

In [ ]:
# 7. 질문하기 함수
def ask_question(query):
    print(f"\n🙋 질문: {query}")
    result = rag_chain.invoke(query)
    print(f"🤖 답변: {result}")

# 테스트
#ask_question("A는 B의 배우자인데 B가 회사를 다니다 간경화 진단을 받아 일을 더이상 못하고 1년전에 퇴사를 하였고 6개월전에 간세포암 진단을 받아 3개월전에 사망하였다. A는 직장 내 괴롭힘, 따돌림, 폭언, 폭행, 사직 강요 등은 없었던 것으로 판명되었다. A는 산업재해보상을 받을수 있을까?")
ask_question("A는 아주 유명한 쿠키 제과점 B의 상표가 아직 상표등록이 안된 것을 알고 자기가 먼저 상표 등록을 하려고 했고, 이를 B가 우연히 알아내고 상표 등록 취하 소송을 했어. 그리고 B는 A에게 저작권 침해 손해배상까지 걸었어. B는 소송에서 이길 수 있을까?")

### 1. 사실관계 비교 분석

질문 상황과 참고 판례의 사실관계를 비교하면 다음과 같습니다.

| 구분 | 질문 상황 (A vs B) | 참고 판례 ([2020허4570]) |
| :--- | :--- | :--- |
| **선사용자 (B)** | 아주 유명한 쿠키 제과점 (상표 미등록 상태) | 피고 (선사용서비스표 권리자) |
| **후출원인 (A)** | B의 유명 상표를 알고 먼저 등록 시도 | 원고 (이 사건 등록서비스표 출원인) |
| **쟁점 1 (상표)** | B의 유명 상표와 동일/유사한 상표를 A가 먼저 등록 시도 | A의 등록서비스표가 B의 선사용서비스표와 유사하며 수요자 오인·혼동 우려 |
| **쟁점 2 (목적)** | A가 B의 상표임을 알고 등록 시도 (부정한 목적 추정) | 등록서비스표가 부정한 목적을 가지고 출원·등록됨 (상표법 제34조 제1항 제13호) |
| **쟁점 3 (저작권)** | B가 A에게 저작권 침해 손해배상 청구 | (직접적인 저작권 침해 쟁점은 없음. 상표법 위반 쟁점만 다룸) |

질문 상황은 **[사건번호: 2020허4570]** 등록무효(상) 사건에서 다루어진 **상표법 제34조 제1항 제12호(수요자 기만 우려)** 및 **제13호(부정한 목적)** 위반 사유와 매우 유사한 상황을 전제하고 있습니다.

### 2. 상표 등록 취소 소송 (상표법적 쟁점) 분석

B가 A의 상표 등록 취소를 구하는 소송에서 승소할 가능성은 매우 높습니다. 이는 주로 상표법 제34조 제1항 제12호 및 제13호에 근거합니다.

#### 가. 수요자 기만 우려 (상표법 제34조 제1항 제12호)

**[2020허4570]** 판결에 따르면, 등록상표가 제12호에 해당하여 무효가 되기 위해서는 등록결정 시를 기준으로 대비되는 선사용상표가 **국내 수요자나 거래자에게 특정인의 상표로 인식될 수 있을 정도로 알려져 있어야** 합니다.

*   **적용:** 질문에서 B의 쿠키 제과점이 **"아주 유명하다"**고 전제하고 있으므로, B의 상표가 등록결정일 당시 국내 수요자에게 특정인의 상표로 인식되었다고 인정될 가능성이 높습니다.
*   **유사성 및 견련성:** A가 등록하려는 상표가 B의 유명 상표와 동일하거나 유사하고, 지정 서비스업(쿠키/제과 관련)이 B의 기존 서비스업과 동일하거나 유사하다면, 수요자는 A의 서비스가 B에 의해 제공되거나 B와 경제적 견련관계가 있는 것으로 오인·혼동할 염려가 있다고 판단될 것입니다 (**[2020허4570]** 판단 근거).

#### 나. 부정한 목적 (상표법 제34조 제1항 제13호)

**[2020허4570]** 판결은 등록서비스표가 **"출원 당시 피고의 상품이나 영업을 나타내는 것으로 수요자들에게 인식되어 있었던 선사용서비스표들과 유사한 서비스표로서, 원고는 이를 부정한 목적을 가지고 출원한 것이므로, 이 사건 등록서비스표는 상표법 제34조 제1항 제13호에 해당한다"**고 명시하고 있습니다.

*   **적용:** A가 B의 상표가 유명하다는 사실을 **"알고"** 먼저 등록을 시도했다는 점은 전형적인 **부정한 목적(Bad Faith)**으로 인정될 수 있습니다. 이는 상표법 제34조 제1항 제13호의 등록무효 사유에 해당하여 등록 취소의 강력한 근거가 됩니다.

### 3. 저작권 침해 손해배상 청구 (저작권법적 쟁점)

질문에서 B가 A에게 **저작권 침해 손해배상**을 청구한다고 하였는데, 참고 판례는 상표법만을 다루고 있어 직접적인 판단 근거는 없습니다. 그러나 다음과 같이 분석할 수 있습니다.

*   **상표와 저작권의 관계:** 상표는 상품의 출처를 표시하는 기능을 하지만, 상표의 디자인 자체가 창작성이 인정될 경우 저작권의 보호 대상이 될 수도 있습니다.
*   **침해 가능성:** 만약 A가 등록을 시도한 상표의 표장(디자인)이 B의 기존 상표 디자인(로고, 캐릭터 등)과 실질적으로 유사하여 저작권법상 복제 또는 2차적 저작물 작성에 해당한다면, B는 저작권 침해를 이유로 손해배상을 청구할 수 있습니다.
*   **결론:** 상표법 위반으로 등록이 무효화될 가능성이 높더라도, 저작권 침해 여부는 별도의 심리 사항이며, B의 상표 디자인에 저작물로서의 창작성이 인정되어야만 손해배상 청구가 인용될 수 있습니다.

### 4. 결론 및 참고 사항

B는 A를 상대로 한 **상표 등록 취소 소송**에서 **매우 높은 승소 가능성**을 가집니다. 이는 A가 B의 유명 상표를 알고 선출원하여 수요자를 기만하거나 부정한 이익을 얻으려 했다는 점이 **[사건번호: 2020허4570]** 판례의 법리에 따라 명확하게 입증될 수 있기 때문입니다.

저작권 침해 손해배상 청구의 승소 여부는 해당 상표 디자인의 구체적인 창작성과 유사성 정도에 따라 별도로 판단될 것입니다.

**참고한 판례:** 특허법원 2021. 04. 27. 선고 **[사건번호: 2020허4570]** 등록무효(상)